In [26]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import json
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, LeaveOneGroupOut, StratifiedGroupKFold
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay
import xgboost as xgb
import warnings
import os

In [27]:
def evaluator(y_pred, y_test, verbose=False):
    """Returns evaluation metric scores"""
    accuracy = accuracy_score(y_pred=y_pred, y_true=y_test)
    balanced_accuracy = balanced_accuracy_score(y_pred=y_pred, y_true=y_test)
    f1 = f1_score(y_pred=y_pred, y_true=y_test, average='weighted')
    recall = recall_score(y_pred=y_pred, y_true=y_test, average='weighted')
    precision = precision_score(y_pred=y_pred, y_true=y_test, average='weighted')
    confusion = confusion_matrix(y_pred=y_pred, y_true=y_test)

    # display scores
    if verbose:
        ConfusionMatrixDisplay(confusion_matrix=confusion, display_labels=[False, True]).plot(cmap=plt.cm.Blues)
        plt.title('Physical fatigue')

        print(f'accuracy: {accuracy}\n'
              f'balanced accuracy: {balanced_accuracy}\n'
              f'f1 (weighted): {f1}\n'
              f'recall (weighted): {recall}\n'
              f'precision (weighted): {precision}')

    return {'accuracy': accuracy,
            'balanced_accuracy': balanced_accuracy,
            'f1': f1,
            'recall': recall,
            'precision': precision,
            'confusion': confusion}

In [28]:
VARIABLES = ['ActivityCounts', 'Barometer', 'BloodPerfusion',
             'BloodPulseWave', 'EnergyExpenditure', 'GalvanicSkinResponse', 'HR',
             'HRV', 'RESP', 'Steps', 'SkinTemperature', 'ActivityClass']

In [29]:
NORMALIZE_TRAIN = True # whether to normalize data acc. to training data
SHUFFLE = True # whether to shuffle data before applying CV

In [30]:
# for reproducability
SEED = 42

# Import data

In [31]:
# file path to data folder
path = './Output'

In [32]:
N, FEATURES = sum([1 for p in os.listdir(path) if p[:19] == 'feature_vector_stat']), \
              *np.load(path + '/feature_vector_stat0.npy').shape
print(f'datapoints: {N}, features: {FEATURES}')

datapoints: 317, features: 284


Feature vector, labels

In [48]:
# init
X = np.empty((N, FEATURES))
y = np.empty((N, 2))

# load individual datapoints
for i in range(N):
    X[i, ] = np.load(path + f'/feature_vector_stat{i}.npy', allow_pickle=True)
    y[i, ] = np.load(path + f'/labels_stat{i}.npy', allow_pickle=True)

Metadata (subjectID etc.)

In [34]:
with open(path + '/metadata_stat.txt') as f:
    metadata = f.read()

metadata = json.loads(metadata.replace('\'', '\"').replace('False', 'false').replace('True', 'true')) # doesn't accept other chars

# XGBoost

In [ ]:
subjects = [meta['subjectID'] for meta in metadata]

### 5-fold stratified CV

In [46]:
# separate label prediction
subj = 23
X = X[[subject == subj for subject in subjects], :]
y = y[[subject == subj for subject in subjects], :]

y_phf, y_mf = y[:, 0], y[:, 1]

y_phf, y_mf = y[:, 0], y[:, 1]

In [47]:
%%time
# nested CV
folds = 5

with warnings.catch_warnings():
    # ignore sklearn warning
    warnings.filterwarnings('ignore')

    for fatigue in ('Physical fatigue', 'Mental fatigue'):
        # load labels
        print(f'Starting cross-validation for {fatigue}')
        y_ = {'Physical fatigue': y_phf, 'Mental fatigue': y_mf}[fatigue] # pick phF or MF

        # CV: performance evaluation
        cv = StratifiedKFold(n_splits=folds, shuffle=True, random_state=SEED) if SHUFFLE \
            else StratifiedKFold(n_splits=folds)
        scores_cv = []
        with tqdm(total=folds) as pbar:
            for i, (train_outer_index, test_outer_index) in enumerate(cv.split(X, y_)):
                # train/test split
                X_train, X_test = X[train_outer_index], X[test_outer_index]
                y_train, y_test = y_[train_outer_index], y_[test_outer_index]

                # normalize features (acc.to train set)
                if NORMALIZE_TRAIN:
                    scaler = StandardScaler()
                    scaler.fit(X_train)
                    X_train = scaler.transform(X_train, copy=True)
                    X_test = scaler.transform(X_test, copy=True)

                # model
                model = xgb.XGBClassifier(random_state=SEED, verbosity=0)

                # training
                model.fit(X_train, y_train)

                # evaluate
                y_pred = model.predict(X_test)
                scores = evaluator(y_pred, y_test, verbose=False)
                scores_cv.append(scores)

                # for progress bar
                pbar.update(1)
                pbar.set_description(f' Fold {i+1} F1: {scores["f1"]}')

        # final evaluation
        print('Performance model:')
        metrics = scores_cv[0].keys()
        for metric in metrics:
            # ignore confusion_matrix
            if metric == 'confusion':
                continue
            mean = np.mean([scores_cv_i[metric] for scores_cv_i in scores_cv])
            std = np.std([scores_cv_i[metric] for scores_cv_i in scores_cv])
            print(f' {metric}: {round(mean, 3)} +- {round(std, 3)} \n')

Starting cross-validation for Physical fatigue


 Fold 5 F1: 0.7868131868131868: 100%|██████████| 5/5 [00:00<00:00, 18.83it/s]


Performance model:
 accuracy: 0.614 +- 0.086 

 balanced_accuracy: 0.612 +- 0.091 

 f1: 0.607 +- 0.09 

 recall: 0.614 +- 0.086 

 precision: 0.621 +- 0.088 

Starting cross-validation for Mental fatigue


 Fold 5 F1: 0.7142857142857143: 100%|██████████| 5/5 [00:00<00:00, 18.12it/s]

Performance model:
 accuracy: 0.629 +- 0.105 

 balanced_accuracy: 0.612 +- 0.112 

 f1: 0.62 +- 0.113 

 recall: 0.629 +- 0.105 

 precision: 0.624 +- 0.12 

CPU times: total: 5.02 s
Wall time: 548 ms


In [41]:
scores_cv

[{'accuracy': 0.6428571428571429,
  'balanced_accuracy': 0.625,
  'f1': 0.6371275783040489,
  'recall': 0.6428571428571429,
  'precision': 0.6380952380952382,
  'confusion': array([[3, 3],
         [2, 6]], dtype=int64)},
 {'accuracy': 0.6428571428571429,
  'balanced_accuracy': 0.6458333333333333,
  'f1': 0.6446886446886447,
  'recall': 0.6428571428571429,
  'precision': 0.6530612244897959,
  'confusion': array([[4, 2],
         [3, 5]], dtype=int64)},
 {'accuracy': 0.42857142857142855,
  'balanced_accuracy': 0.3958333333333333,
  'f1': 0.4031746031746032,
  'recall': 0.42857142857142855,
  'precision': 0.39285714285714285,
  'confusion': array([[1, 5],
         [3, 5]], dtype=int64)},
 {'accuracy': 0.7142857142857143,
  'balanced_accuracy': 0.6875,
  'f1': 0.7015873015873015,
  'recall': 0.7142857142857143,
  'precision': 0.7214285714285714,
  'confusion': array([[3, 3],
         [1, 7]], dtype=int64)},
 {'accuracy': 0.7142857142857143,
  'balanced_accuracy': 0.7083333333333333,
  'f1

In [43]:
# separate label prediction
subj = 24
X = X[[subject == subj for subject in subjects], :]
y = y[[subject == subj for subject in subjects], :]

y_phf, y_mf = y[:, 0], y[:, 1]

In [44]:
%%time
# nested CV
folds = 5

with warnings.catch_warnings():
    # ignore sklearn warning
    warnings.filterwarnings('ignore')

    for fatigue in ('Physical fatigue', 'Mental fatigue'):
        # load labels
        print(f'Starting cross-validation for {fatigue}')
        y_ = {'Physical fatigue': y_phf, 'Mental fatigue': y_mf}[fatigue] # pick phF or MF

        # CV: performance evaluation
        cv = StratifiedKFold(n_splits=folds, shuffle=True, random_state=SEED) if SHUFFLE \
            else StratifiedKFold(n_splits=folds)
        scores_cv = []
        with tqdm(total=folds) as pbar:
            for i, (train_outer_index, test_outer_index) in enumerate(cv.split(X, y_)):
                # train/test split
                X_train, X_test = X[train_outer_index], X[test_outer_index]
                y_train, y_test = y_[train_outer_index], y_[test_outer_index]

                # normalize features (acc.to train set)
                if NORMALIZE_TRAIN:
                    scaler = StandardScaler()
                    scaler.fit(X_train)
                    X_train = scaler.transform(X_train, copy=True)
                    X_test = scaler.transform(X_test, copy=True)

                # model
                model = xgb.XGBClassifier(random_state=SEED, verbosity=0)

                # training
                model.fit(X_train, y_train)

                # evaluate
                y_pred = model.predict(X_test)
                scores = evaluator(y_pred, y_test, verbose=False)
                scores_cv.append(scores)

                # for progress bar
                pbar.update(1)
                pbar.set_description(f' Fold {i+1} F1: {scores["f1"]}')

        # final evaluation
        print('Performance model:')
        metrics = scores_cv[0].keys()
        for metric in metrics:
            # ignore confusion_matrix
            if metric == 'confusion':
                continue
            mean = np.mean([scores_cv_i[metric] for scores_cv_i in scores_cv])
            std = np.std([scores_cv_i[metric] for scores_cv_i in scores_cv])
            print(f' {metric}: {round(mean, 3)} +- {round(std, 3)} \n')

Starting cross-validation for Physical fatigue


 Fold 5 F1: 0.9217638691322904: 100%|██████████| 5/5 [00:00<00:00, 20.70it/s]


Performance model:
 accuracy: 0.947 +- 0.033 

 balanced_accuracy: 0.594 +- 0.203 

 f1: 0.932 +- 0.036 

 recall: 0.947 +- 0.033 

 precision: 0.917 +- 0.041 

Starting cross-validation for Mental fatigue


 Fold 5 F1: 0.7430340557275542: 100%|██████████| 5/5 [00:00<00:00, 18.80it/s]

Performance model:
 accuracy: 0.821 +- 0.054 

 balanced_accuracy: 0.481 +- 0.025 

 f1: 0.768 +- 0.043 

 recall: 0.821 +- 0.054 

 precision: 0.723 +- 0.039 

CPU times: total: 4.78 s
Wall time: 514 ms


In [49]:
# separate label prediction
subj = 26
X = X[[subject == subj for subject in subjects], :]
y = y[[subject == subj for subject in subjects], :]

y_phf, y_mf = y[:, 0], y[:, 1]

In [50]:
%%time
# nested CV
folds = 5

with warnings.catch_warnings():
    # ignore sklearn warning
    warnings.filterwarnings('ignore')

    for fatigue in ('Physical fatigue', 'Mental fatigue'):
        # load labels
        print(f'Starting cross-validation for {fatigue}')
        y_ = {'Physical fatigue': y_phf, 'Mental fatigue': y_mf}[fatigue] # pick phF or MF

        # CV: performance evaluation
        cv = StratifiedKFold(n_splits=folds, shuffle=True, random_state=SEED) if SHUFFLE \
            else StratifiedKFold(n_splits=folds)
        scores_cv = []
        with tqdm(total=folds) as pbar:
            for i, (train_outer_index, test_outer_index) in enumerate(cv.split(X, y_)):
                # train/test split
                X_train, X_test = X[train_outer_index], X[test_outer_index]
                y_train, y_test = y_[train_outer_index], y_[test_outer_index]

                # normalize features (acc.to train set)
                if NORMALIZE_TRAIN:
                    scaler = StandardScaler()
                    scaler.fit(X_train)
                    X_train = scaler.transform(X_train, copy=True)
                    X_test = scaler.transform(X_test, copy=True)

                # model
                model = xgb.XGBClassifier(random_state=SEED, verbosity=0)

                # training
                model.fit(X_train, y_train)

                # evaluate
                y_pred = model.predict(X_test)
                scores = evaluator(y_pred, y_test, verbose=False)
                scores_cv.append(scores)

                # for progress bar
                pbar.update(1)
                pbar.set_description(f' Fold {i+1} F1: {scores["f1"]}')

        # final evaluation
        print('Performance model:')
        metrics = scores_cv[0].keys()
        for metric in metrics:
            # ignore confusion_matrix
            if metric == 'confusion':
                continue
            mean = np.mean([scores_cv_i[metric] for scores_cv_i in scores_cv])
            std = np.std([scores_cv_i[metric] for scores_cv_i in scores_cv])
            print(f' {metric}: {round(mean, 3)} +- {round(std, 3)} \n')

Starting cross-validation for Physical fatigue


 Fold 5 F1: 1.0: 100%|██████████| 5/5 [00:00<00:00, 20.20it/s]               


Performance model:
 accuracy: 0.98 +- 0.04 

 balanced_accuracy: 0.9 +- 0.2 

 f1: 0.971 +- 0.059 

 recall: 0.98 +- 0.04 

 precision: 0.962 +- 0.076 

Starting cross-validation for Mental fatigue


 Fold 5 F1: 0.8366013071895425: 100%|██████████| 5/5 [00:00<00:00, 20.28it/s]

Performance model:
 accuracy: 0.938 +- 0.051 

 balanced_accuracy: 0.7 +- 0.245 

 f1: 0.908 +- 0.075 

 recall: 0.938 +- 0.051 

 precision: 0.882 +- 0.097 

CPU times: total: 4.11 s
Wall time: 502 ms
